# Preprocessing data Youtube Rewind 2018

#### Import library

In [1]:
from dotenv import load_dotenv
from pymongo import MongoClient
import os
import pandas as pd
import re

In [2]:
# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

In [3]:
database_name = "youtube_rewind_indonesia"
collection_from = "cleansing_2018"
collection_name = "preprocessing_2018"

#### Import dataset dari MongoDB

In [4]:
# Koneksi ke MongoDB
client = MongoClient(local_url)

# Pilih database dan koleksi
db = client[database_name]
collection = db[collection_from]

# Query data dari koleksi
cursor = collection.find()

# Konversi cursor ke list dan kemudian ke DataFrame
data = list(cursor)
df = pd.DataFrame(data)

# Menutup koneksi
client.close()

In [5]:
df

,_id,textOriginal
0,6651606257713e470060633e,Hellow
1,6651606257713e470060633f,Kangenn Dehh.. 🥶😭
2,6651606257713e4700606340,Dari 2024 😄
3,6651606257713e4700606341,Baru nonton di tahun 2024
4,6651606257713e4700606342,2018 ternyata 6th lalu
...,...,...
92033,6651619157713e470062169b,Yuhuuuuuuu
92034,6651619157713e470062169c,Mantulll:)
92035,6651619157713e470062169f,Di like dlu pasti nya
92036,6651619157713e47006216a0,Yt rewind indo 2018 sangat keren !!!\nSangat b...


#### Mengubah data text menjadi lowercase

In [6]:
df['textOriginal'] = df['textOriginal'].str.lower()

df

,_id,textOriginal
0,6651606257713e470060633e,hellow
1,6651606257713e470060633f,kangenn dehh.. 🥶😭
2,6651606257713e4700606340,dari 2024 😄
3,6651606257713e4700606341,baru nonton di tahun 2024
4,6651606257713e4700606342,2018 ternyata 6th lalu
...,...,...
92033,6651619157713e470062169b,yuhuuuuuuu
92034,6651619157713e470062169c,mantulll:)
92035,6651619157713e470062169f,di like dlu pasti nya
92036,6651619157713e47006216a0,yt rewind indo 2018 sangat keren !!!\nsangat b...


#### Menghapus data dengan atribut text null

In [7]:
df = df[df['textOriginal'] != '']
df

,_id,textOriginal
0,6651606257713e470060633e,hellow
1,6651606257713e470060633f,kangenn dehh.. 🥶😭
2,6651606257713e4700606340,dari 2024 😄
3,6651606257713e4700606341,baru nonton di tahun 2024
4,6651606257713e4700606342,2018 ternyata 6th lalu
...,...,...
92033,6651619157713e470062169b,yuhuuuuuuu
92034,6651619157713e470062169c,mantulll:)
92035,6651619157713e470062169f,di like dlu pasti nya
92036,6651619157713e47006216a0,yt rewind indo 2018 sangat keren !!!\nsangat b...


#### Menghapus baris dengan nilai yang hanya berupa numeric pada kolom 'text'

In [8]:
df = df[~df['textOriginal'].str.isnumeric()]

df

,_id,textOriginal
0,6651606257713e470060633e,hellow
1,6651606257713e470060633f,kangenn dehh.. 🥶😭
2,6651606257713e4700606340,dari 2024 😄
3,6651606257713e4700606341,baru nonton di tahun 2024
4,6651606257713e4700606342,2018 ternyata 6th lalu
...,...,...
92033,6651619157713e470062169b,yuhuuuuuuu
92034,6651619157713e470062169c,mantulll:)
92035,6651619157713e470062169f,di like dlu pasti nya
92036,6651619157713e47006216a0,yt rewind indo 2018 sangat keren !!!\nsangat b...


#### Fungsi untuk membersihkan teks dari beberapa pola yang umum ditemukan dalam data teks

In [9]:
def clean_data_text(text):
    # Menghapus username
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Menghapus hashtag
    text = re.sub(r'#\w+', '', text)
    # Menghapus angka
    text = re.sub(r'\d+', '', text)
    # Menghapus 'RT' (retweet)
    text = re.sub(r'RT[\s]+', '', text)
    # Menghapus URL
    text = re.sub(r'https?://\S+', '', text)
    # Menghapus karakter non-alfanumerik
    text = re.sub(r'[^A-Za-z0-9 ]', '', text)
    # Menghapus spasi ekstra di awal dan akhir teks, serta spasi berlebih di antara kata
    text = re.sub(r'\s+', ' ', text).strip()
    # Mencari dan menghapus karakter tunggal yang terpisah dari kata-kata
    text = re.sub(r'\b\w\b', '', text)
    # Menghapus angka tunggal
    text = re.sub(r'\b\d+\b', '', text)
    # Menghapus karakter berulang, misalnya: 'aa', 'bbb', 'ccc'
    text = re.sub(r'(.)\1+', r'\1', text)
    # Menghapus emotikon
    text = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)
    return text

In [10]:
# Apply clean_data_text pada dataframe
# Menggunakan .loc untuk mengakses DataFrame
df.loc[:, 'textOriginal'] = df['textOriginal'].apply(clean_data_text)

df

,_id,textOriginal
0,6651606257713e470060633e,helow
1,6651606257713e470060633f,kangen deh
2,6651606257713e4700606340,dari
3,6651606257713e4700606341,baru nonton di tahun
4,6651606257713e4700606342,ternyata th lalu
...,...,...
92033,6651619157713e470062169b,yuhu
92034,6651619157713e470062169c,mantul
92035,6651619157713e470062169f,di like dlu pasti nya
92036,6651619157713e47006216a0,yt rewind indo sangat keren sangat bermakna sa...


#### Fungsi untuk membersihkan teks dari kata yang hanya 1 huruf

In [11]:
def remove_single_letter_words(text):
    return re.sub(r'\b\w\b', '', text).strip()

In [12]:
# Apply remove_single_letter_words pada dataframe
# Menggunakan .loc untuk mengakses DataFrame
df.loc[:, 'textOriginal'] = df['textOriginal'].apply(remove_single_letter_words)

df

,_id,textOriginal
0,6651606257713e470060633e,helow
1,6651606257713e470060633f,kangen deh
2,6651606257713e4700606340,dari
3,6651606257713e4700606341,baru nonton di tahun
4,6651606257713e4700606342,ternyata th lalu
...,...,...
92033,6651619157713e470062169b,yuhu
92034,6651619157713e470062169c,mantul
92035,6651619157713e470062169f,di like dlu pasti nya
92036,6651619157713e47006216a0,yt rewind indo sangat keren sangat bermakna sa...


#### Memeriksa panjang teks pada kolom 'text' dan menyimpan hanya baris dengan panjang antara 5 sampai 200 karakter

In [13]:
df = df[(df['textOriginal'].str.len() >= 5) & (df['textOriginal'].str.len() <= 200)]

df

,_id,textOriginal
0,6651606257713e470060633e,helow
1,6651606257713e470060633f,kangen deh
3,6651606257713e4700606341,baru nonton di tahun
4,6651606257713e4700606342,ternyata th lalu
5,6651606257713e4700606343,era sebelum tiktoker menyerang
...,...,...
92031,6651619157713e4700621697,yang stay menungusampe bolak balik klik nungui...
92034,6651619157713e470062169c,mantul
92035,6651619157713e470062169f,di like dlu pasti nya
92036,6651619157713e47006216a0,yt rewind indo sangat keren sangat bermakna sa...


### Memilih hanya 10000 data sebagai sampel dari keseluruhan dataset yang telah bersih

In [14]:
# Memilih 10.000 data secara acak
df_sampled = df.sample(n=10000, random_state=42)

df_sampled

,_id,textOriginal
3376,6651606d57713e4700607159,di chanel gw ada chef renata maen ome tv
6353,6651607557713e4700607de5,bang erwin
49805,665160fc57713e4700613fea,musik producernya emang harus eka kacaw ini as...
74312,6651614c57713e470061b2c9,tu daylen teriak teriak kayak tai
87736,6651617f57713e470061fc0a,biasa sa cinta mana
...,...,...
81581,6651616657713e470061d685,agak ngak sesuai ekspektasi sih
59452,6651611b57713e4700616d2a,keren tapi kerenan menurut gw
51475,6651610157713e47006147ba,kenapa nga ada scene kimihime lagi renang pada...
69652,6651613c57713e4700619d1d,buat creator mantep dah


### Normalisasi

In [15]:
norm = {
    " yg ": " yang ",
    " kren ": " keren",
    " sdh ": " sudah ",
    " dgn ": " dengan ",
    " tdk ": " tidak ",
    " dlm ": " dalam ",
    " dl ": " dulu ",
    " gw ": " saya ",
    " kok ": " kenapa ",
    " bs ": " bisa ",
    " bgt ": " sangat ",
    " njir ": " kagum ",
    " anjir ": " kagum ",
    " g ": " tidak ",
    " lg ": " lagi ",
    " dr ": " dari ",
    " tp ": " tapi ",
    " jg ": " juga ",
    " sm ": " sama ",
    " krn ": " karena ",
    " spt ": " seperti ",
    " yg ": " yang ",
    " pd ": " pada ",
    " dn ": " dan ",
    " d ": " di ",
    " kt ": " kita ",
    " km ": " kamu ",
    " ms ": " masih ",
    " tlg ": " tolong ",
    " sja ": " saja ",
    " sy ": " saya ",
    " sdikit ": " sedikit ",
    " dket ": " dekat ",
    " lbh ": " lebih ",
    " sj ": " saja ",
    " bnyk ": " banyak ",
    " dgnnya ": " dengannya ",
    " sgala ": " segala ",
    " skrg ": " sekarang ",
    " nmr ": " nomor ",
    " srg ": " sering ",
    " ttg ": " tentang ",
    " smua ": " semua ",
    " trus ": " terus ",
    " pdhl ": " padahal ",
    " kt ": " kita ",
    " dkt ": " dekat ",
    " jg ": " juga ",
    " dlm ": " dalam ",
    " bhw ": " bahwa ",
    " sja ": " saja ",
    " mrk ": " mereka ",
    " sgth ": " sangat ",
    " dn ": " dan ",
    " trlalu ": " terlalu ",
    " msing2 ": " masing-masing ",
    " brp ": " berapa ",
    " aq ": " aku ",
    " smpe ": " sampai ",
    " kalo ": " kalau ",
    " bkn ": " bukan ",
    " jd ": " jadi ",
    " dket ": " dekat ",
    " ntr ": " nanti ",
    " klo ": " kalau ",
    " sda ": " sudah ",
    " sy ": " saya ",
    " kpn ": " kapan ",
    " dmn ": " dimana ",
    " lgsng ": " langsung ",
    " tdk ": " tidak ",
    " pgi ": " pagi ",
    " ngga ": " tidak ",
    " gak ": " tidak ",
    " lbih ": " lebih ",
    " cm ": " cuman ",
    " sgt ": " sangat ",
    " aj ": " saja ",
    " apa2 ": " apa-apa ",
    " krg ": " kurang ",
    " ngerti ": " mengerti ",
    " slalu ": " selalu ",
    " bbrp ": " beberapa ",
    " smpe ": " sampai ",
    " krja ": " kerja ",
    " bbrp ": " beberapa ",
    " jln ": " jalan ",
    " hr ": " hari ",
    " kok ": " kenapa ",
    " mgkin ": " mungkin "
    # Tambahkan aturan normalisasi lainnya di sini sesuai kebutuhan
}

#### Fungsi normalisasi

In [16]:
def normalisasi(text):
    for i in norm:
        text = text.replace(i, norm[i])
    return text

In [17]:
df_sampled.loc[:, 'textOriginal'] = df_sampled['textOriginal'].apply(normalisasi)

df_sampled

,_id,textOriginal
3376,6651606d57713e4700607159,di chanel saya ada chef renata maen ome tv
6353,6651607557713e4700607de5,bang erwin
49805,665160fc57713e4700613fea,musik producernya emang harus eka kacaw ini as...
74312,6651614c57713e470061b2c9,tu daylen teriak teriak kayak tai
87736,6651617f57713e470061fc0a,biasa sa cinta mana
...,...,...
81581,6651616657713e470061d685,agak ngak sesuai ekspektasi sih
59452,6651611b57713e4700616d2a,keren tapi kerenan menurut gw
51475,6651610157713e47006147ba,kenapa nga ada scene kimihime lagi renang pada...
69652,6651613c57713e4700619d1d,buat creator mantep dah


## Stopwords

In [18]:
import Sastrawi

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

more_stop_words = ["tidak"]

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text

In [19]:
df_sampled.loc[:, 'textOriginal'] = df_sampled['textOriginal'].apply(stopword)

df_sampled

,_id,textOriginal
3376,6651606d57713e4700607159,chanel ada chef renata maen ome tv
6353,6651607557713e4700607de5,bang erwin
49805,665160fc57713e4700613fea,musik producernya emang eka kacaw asli kece pa...
74312,6651614c57713e470061b2c9,tu daylen teriak teriak kayak tai
87736,6651617f57713e470061fc0a,biasa sa cinta mana
...,...,...
81581,6651616657713e470061d685,ngak sesuai ekspektasi sih
59452,6651611b57713e4700616d2a,keren kerenan gw
51475,6651610157713e47006147ba,nga scene kimihime renang padahal kan salah sa...
69652,6651613c57713e4700619d1d,buat creator mantep dah


## Tokenisasi
#### Memisahkan kalimat menjadi kata-kata

In [20]:
tokenized = df_sampled['textOriginal'].apply(lambda x:x.split())
tokenized

3376            [chanel, ada, chef, renata, maen, ome, tv]
6353                                         [bang, erwin]
49805    [musik, producernya, emang, eka, kacaw, asli, ...
74312             [tu, daylen, teriak, teriak, kayak, tai]
87736                             [biasa, sa, cinta, mana]
                               ...                        
81581                      [ngak, sesuai, ekspektasi, sih]
59452                                 [keren, kerenan, gw]
51475    [nga, scene, kimihime, renang, padahal, kan, s...
69652                         [buat, creator, mantep, dah]
43348    [kebanyakan, nonton, nyari, kesalahanya, doang...
Name: textOriginal, Length: 10000, dtype: object

## Stemming
#### Mengubah kata berimbuhan menjadi kata dasar

In [21]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi counter
counter = 1

def stemming(text_cleaning):
    global counter
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in text_cleaning:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(f"{counter}: {d_clean}") 
    counter += 1
    return d_clean

Proses berikut agak lama karena data besar

In [22]:
tokenized = tokenized.apply(stemming)
# tokenized.to_csv("tokenized_comments.csv", index=False)
tokenized

1: chanel ada chef renata maen ome tv
2: bang erwin
3: musik producernya emang eka kacaw asli kece parah musik not so god
4: tu daylen teriak teriak kayak tai
5: biasa sa cinta mana
6: siapa kesini abis nonton rewind
7: bagus sih lebih tarik sama
8: yutub rewind keren dah inti
9: congrats
10: ada si bowongacawin aja
11: ini kata sponsor baru liat gw
12: wkwkwk younglex anti eka gustiwana
13: wah tambah keren bingit mantap sekali bydwy kece
14: waduh
15: naha btn
16: ria ricis ga
17: gk ad miawaug
18: awal ubah kualitas youtube indonesia
19: keren sih kayak nya musik nya terlalu monoton
20: pesa nyambung jelas tidak kaya yt rewind pesa nyampe
21: keren eriko lim banga
22: salut gua bangindonesia jaya
23: yg ngedislike tau fans mro
24: emang bowo yutuber
25: kalau semua datang keren takut pusing wkwkw udh keren gomong erpan bikers ngapain woi
26: entah kalau petir malas nonton mungkin kerna isi konten nya pamer vlok mulu
27: tingal nonton aja bacot abis like dislike udah diem napa netize

3376                    chanel ada chef renata maen ome tv
6353                                            bang erwin
49805    musik producernya emang eka kacaw asli kece pa...
74312                    tu daylen teriak teriak kayak tai
87736                                  biasa sa cinta mana
                               ...                        
81581                           ngak sesuai ekspektasi sih
59452                                       keren keren gw
51475    nga scene kimihime renang padahal kan salah sa...
69652                              buat creator mantep dah
43348    banyak nonton nyari kesalahanya doang kek kont...
Name: textOriginal, Length: 10000, dtype: object

In [23]:
tokenized

3376                    chanel ada chef renata maen ome tv
6353                                            bang erwin
49805    musik producernya emang eka kacaw asli kece pa...
74312                    tu daylen teriak teriak kayak tai
87736                                  biasa sa cinta mana
                               ...                        
81581                           ngak sesuai ekspektasi sih
59452                                       keren keren gw
51475    nga scene kimihime renang padahal kan salah sa...
69652                              buat creator mantep dah
43348    banyak nonton nyari kesalahanya doang kek kont...
Name: textOriginal, Length: 10000, dtype: object

#### Mengimpan Tokenized Comments ke MongoDB Local

In [24]:
# Koneksi ke MongoDB
client = MongoClient(local_url)
db = client[database_name]
collection = db[collection_name]

# Konversi tokenized menjadi list of dictionaries
data_to_insert = [{'textOriginal': text} for text in tokenized]

# Masukkan data ke dalam koleksi MongoDB
collection.insert_many(data_to_insert)

print("Data berhasil diimpor ke MongoDB Local.")

Data berhasil diimpor ke MongoDB Local.


## Translate

import data dari file csv

In [25]:
# from translate import Translator

# # Inisialisasi counter
# counter = 1

# def convert_eng(text):
#     global counter  # Menggunakan counter sebagai variabel global
#     translator = Translator(to_lang="en", from_lang="id")
#     translation = translator.translate(text)
#     print(f"{counter}: {translation}")  # Mencetak teks terjemahan dengan nomor urut
#     counter += 1  # Menambah nilai counter setiap kali fungsi dipanggil
#     return translation

In [26]:
# # data["tweet_english"] = data["text"].apply(convert_eng)
# data.to_csv("translate_comments.csv", index=False)